Shivansh Chaturvedi

Advanced Machine Learning HW3

GitHub Repo: https://github.com/shivansh-chaturvedi/GR5074-Advanced-Machine-Learning

In [ ]:
#install aimodelshare library
! pip install aimodelshare==0.0.189


In [ ]:
# imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from zipfile import ZipFile

from mpl_toolkits.axes_grid1 import ImageGrid
import random

import sys
import time
import cv2
from matplotlib import pyplot as plt
import tensorflow as tf
import os

from sklearn.model_selection import train_test_split

from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from tensorflow.python.keras.layers.convolutional import Conv2D, MaxPooling2D 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.applications import VGG19, ResNet50, InceptionV3

from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten
from tensorflow.keras import backend as K

import aimodelshare as ai

In [ ]:
#import data

# Get competition data
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/sst2_competition_data-repository:latest') 

# Set up X_train, X_test, and y_train_labels objects
import warnings
warnings.simplefilter(action='ignore', category=Warning)

X_train=pd.read_csv("sst2_competition_data/X_train.csv", squeeze=True)
X_test=pd.read_csv("sst2_competition_data/X_test.csv", squeeze=True)

y_train_labels=pd.read_csv("sst2_competition_data/y_train_labels.csv", squeeze=True)

# ohe encode Y data
y_train = pd.get_dummies(y_train_labels)


Data downloaded successfully.


In [ ]:
#Set credentials using modelshare.org username/password
from aimodelshare.aws import set_credentials
import aimodelshare as ai

apiurl="https://rlxjxnoql9.execute-api.us-east-1.amazonaws.com/prod/m"
set_credentials(apiurl=apiurl)

#Instantiate Competition
mycompetition = ai.Competition(apiurl)
     

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [ ]:
#preprocessing

# This preprocessor function makes use of the tf.keras tokenizer

from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

# Build vocabulary from training text data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)

# preprocessor tokenizes words and makes sure all documents have the same length
def preprocessor(data, maxlen=40, max_words=10000):

    sequences = tokenizer.texts_to_sequences(data)

    word_index = tokenizer.word_index
    X = pad_sequences(sequences, maxlen=maxlen)

    return X

print(preprocessor(X_train).shape)
print(preprocessor(X_test).shape)

(6920, 40)
(1821, 40)


In [ ]:
ai.export_preprocessor(preprocessor,"") 


Your preprocessor is now saved to 'preprocessor.zip'


### (1) Discuss the dataset in general terms and describe why building a predictive model using this data might be practically useful.  Who could benefit from a model like this? Explain.

This is a dataset of movie reviews, which can be broadly classified into positive or negative sentiments. Analysis of this data could be helpful for publications seeking to describe the reception of a movie, studios looking to asses the performance of their investments, and other critics to gauge how their peers reviewed the work.

### (2) Run at least three prediction models to try to predict the SST sentiment dataset well.

* Use an Embedding layer and LSTM layers in at least one model
* Use an Embedding layer and Conv1d layers in at least one model
* Use transfer learning with glove embeddings for at least one of these models
* Discuss which models performed better and point out relevant hyper-parameter values for successful models.
*Submit your best three models to the leader board for the SST Model Share competition.

In [ ]:
# embedding + LSTM

In [ ]:
# Train and submit model 2 using same preprocessor (note that you could save a new preprocessor, but we will use the same one for this example).
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Flatten

model2 = Sequential()
model2.add(Embedding(10000, 16, input_length=40))
model2.add(LSTM(32, return_sequences=True, dropout=0.2))
model2.add(LSTM(32, dropout=0.2))
model2.add(Flatten())
model2.add(Dense(2, activation='softmax'))

model2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

history = model2.fit(preprocessor(X_train), y_train,
                    epochs=2,
                    batch_size=15,
                    validation_split=0.2)

Epoch 1/2
370/370 [==============================] - 17s 37ms/step - loss: 0.6472 - acc: 0.6299 - val_loss: 0.9359 - val_acc: 0.3678
Epoch 2/2
370/370 [==============================] - 16s 44ms/step - loss: 0.4914 - acc: 0.7668 - val_loss: 1.4992 - val_acc: 0.3194


In [ ]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model2, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model2.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [ ]:
#submit model

#-- Generate predicted y values (Model 1)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=model2.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model2.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels,
                           custom_metadata = {'team': 3})

57/57 [==============================] - 1s 9ms/step
Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 334

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


In [ ]:
# embedding + conv1d

In [ ]:
from tensorflow.keras.layers import Dense, Embedding,Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import SimpleRNN, LSTM,Embedding

model3 = Sequential()
model3.add(Embedding(10000, 16, input_length=40))
model3.add(layers.Conv1D(16, 4, activation='relu')) 
model3.add(layers.MaxPooling1D(8))
model3.add(layers.Conv1D(16, 4, activation='relu'))
model3.add(layers.GlobalMaxPooling1D())
model3.add(layers.Dense(4))
model3.add(Flatten())
model3.add(Dense(2, activation='softmax'))
model3.add(Dense(2, activation='softmax'))
model3.add(Dense(2, activation='softmax'))
model3.summary()

model3.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

history = model3.fit(preprocessor(X_train), y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 40, 16)            160000    
                                                                 
 conv1d (Conv1D)             (None, 37, 16)            1040      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 4, 16)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1, 16)             1040      
                                                                 
 global_max_pooling1d (Globa  (None, 16)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense_2 (Dense)             (None, 4)                

In [ ]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model3, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model3.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [ ]:

# Submit model

#-- Generate predicted y values
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=model3.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model3.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels,
                           custom_metadata = {'team': 3})

57/57 [==============================] - 0s 1ms/step
Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 335

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


In [ ]:
# transfer learning + glove embedding

In [ ]:
! wget http://nlp.stanford.edu/data/wordvecs/glove.6B.zip

--2023-04-18 00:06:15--  http://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/wordvecs/glove.6B.zip [following]
--2023-04-18 00:06:15--  https://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip [following]
--2023-04-18 00:06:16--  https://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182753 (822M) [app

In [ ]:
! unzip glove.6B.zip 

Archive:  glove.6B.zip
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       
  inflating: glove.6B.50d.txt        


In [ ]:
# Extract embedding data for 100 feature embedding matrix
import os
glove_dir = os.getcwd()

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

# Tokenize the data into one hot vectors
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 150  # We will cut reviews after 150 words in sequence
training_samples = 10000  # We will be training on 10000 samples
validation_samples = 10000  # We will be validating on 10000 samples
max_words = 5000  # We will only consider the top 5000 words in the dataset

Found 400001 word vectors.


In [ ]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

sequences = tokenizer.texts_to_sequences(X_train) # converts words in each text to each word's numeric index in tokenizer dictionary.

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)
labels = np.asarray(y_train['Negative'])

print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
     

Found 13835 unique tokens.
Shape of data tensor: (6920, 150)
Shape of label tensor: (6920,)


In [ ]:
# Split the data into a training set and a validation set
# But first, shuffle the data, since we started from data
# where sample are ordered (all negative first, then all positive).
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

X_train_1 = data[:training_samples] #100 words
y_train_1 = labels[:training_samples]
X_val_1 = data[training_samples: training_samples + validation_samples]
y_val_1 = labels[training_samples: training_samples + validation_samples]
     

In [ ]:

# Build embedding matrix
embedding_dim = 100 # change if you use txt files using larger number of features

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
     

In [ ]:

# Set up same model architecture as before and then import Glove weights to Embedding layer:
from tensorflow.keras.layers import Dense, Embedding,Flatten
from tensorflow.keras.models import Sequential
import tensorflow as tf

model4 = tf.keras.Sequential()
model4.add(tf.keras.layers.Embedding(max_words, embedding_dim, input_length=maxlen))
model4.add(tf.keras.layers.Flatten())
model4.add(tf.keras.layers.Dense(32, activation='relu'))
model4.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model4.summary()

# Add weights in same manner as transfer learning and turn of trainable option before fitting model to freeze weights.
model4.layers[0].set_weights([embedding_matrix])
model4.layers[0].trainable = False

model4.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model4.fit(X_train_1, y_train_1,
                    epochs=5,
                    batch_size=32,
                    validation_data=(X_val_1, y_val_1))
model4.save_weights('pre_trained_glove_model.h5')


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 150, 100)          500000    
                                                                 
 flatten_4 (Flatten)         (None, 15000)             0         
                                                                 
 dense_8 (Dense)             (None, 32)                480032    
                                                                 
 dense_9 (Dense)             (None, 1)                 33        
                                                                 
Total params: 980,065
Trainable params: 980,065
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
217/217 [==============================] - 2s 5ms/step - loss: 0.6429 - acc: 0.6263
Epoch 2/5
217/217 [==============================] - 1s 5ms/step - loss: 0.5399 - acc:

In [ ]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model4, framework='keras',
                          transfer_learning=True,
                          deep_learning=True)

with open("model4.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [ ]:
# submit model

#using tokenizer object we fit to test data above
sequences = tokenizer.texts_to_sequences(X_test)
X_test_1 = pad_sequences(sequences, maxlen=maxlen)

#-- Generate predicted y values (Model 1)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=model4.predict(X_test_1).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model4.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels,
                           custom_metadata = {'team': 3})

57/57 [==============================] - 0s 2ms/step
Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 336

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


### (3) After you submit your first three models, describe your best model with your team via your team slack channel
* Fit and submit up to three more models after learning from your team.
* Discuss results

My best model was embedding layer + LSTM: two LSTM layers and 0.2 dropout, 2 epochs — accuracy of 77.8%

In [ ]:

# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

my_data = data[data['username'] == 'shivanshc']

# Stylize leaderboard data
mycompetition.stylize_leaderboard(my_data)

,accuracy,f1_score,precision,recall,ml_framework,transfer_learning,deep_learning,model_type,depth,num_params,embedding_layers,conv1d_layers,maxpooling1d_layers,simplernn_layers,dropout_layers,flatten_layers,lstm_layers,inputlayer_layers,concatenate_layers,bidirectional_layers,globalmaxpooling1d_layers,globalaveragepooling1d_layers,dense_layers,batchnormalization_layers,sigmoid_act,softmax_act,tanh_act,relu_act,loss,optimizer,memory_size,team,username,version
115,77.83%,77.77%,78.14%,77.83%,keras,nan,True,Sequential,5.000000,174658.000000,1.000000,nan,nan,nan,nan,1.000000,2.000000,nan,nan,nan,nan,nan,1.000000,nan,nan,1.000000,2.000000,nan,str,RMSprop,699936.000000,3.000000,shivanshc,330
236,68.06%,66.93%,70.94%,68.08%,keras,nan,True,Sequential,10.000000,162170.000000,1.000000,2.000000,1.000000,nan,nan,1.000000,nan,nan,nan,nan,1.000000,nan,4.000000,nan,nan,3.000000,nan,2.000000,str,RMSprop,650480.000000,3.000000,shivanshc,335
245,60.15%,53.79%,72.77%,60.19%,keras,nan,True,Sequential,5.000000,174658.000000,1.000000,nan,nan,nan,nan,1.000000,2.000000,nan,nan,nan,nan,nan,1.000000,nan,nan,1.000000,2.000000,nan,str,RMSprop,699936.000000,3.000000,shivanshc,334
304,50.05%,33.36%,25.03%,50.00%,keras,True,True,Sequential,4.000000,980065.000000,1.000000,nan,nan,nan,nan,1.000000,nan,nan,nan,nan,nan,nan,2.000000,nan,1.000000,nan,nan,1.000000,str,RMSprop,3921008.000000,3.000000,shivanshc,336


My team shared other ways to be successful, which I used to inform my other models

In [ ]:
# increasing my epochs for my best model

In [ ]:
# Train and submit model 5 using same preprocessor (note that you could save a new preprocessor, but we will use the same one for this example).
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Flatten

model2v2 = Sequential()
model2v2.add(Embedding(10000, 16, input_length=40))
model2v2.add(LSTM(32, return_sequences=True, dropout=0.2))
model2v2.add(LSTM(32, dropout=0.2))
model2v2.add(Flatten())
model2v2.add(Dense(2, activation='softmax'))

model2v2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

history = model2v2.fit(preprocessor(X_train), y_train,
                    epochs=5, # note increased epochs
                    batch_size=15,
                    validation_split=0.2)


onnx_model = model_to_onnx(model2v2, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model2v2.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

    #submit model

#-- Generate predicted y values (Model 1)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=model2.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model2v2.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels,
                           custom_metadata = {'team': 3})

Epoch 1/5
370/370 [==============================] - 17s 37ms/step - loss: 0.6412 - acc: 0.6384 - val_loss: 0.6522 - val_acc: 0.6308
Epoch 2/5
370/370 [==============================] - 12s 33ms/step - loss: 0.4777 - acc: 0.7767 - val_loss: 0.5375 - val_acc: 0.7616
Epoch 3/5
370/370 [==============================] - 12s 33ms/step - loss: 0.3997 - acc: 0.8228 - val_loss: 0.5144 - val_acc: 0.7702
Epoch 4/5
370/370 [==============================] - 11s 31ms/step - loss: 0.3520 - acc: 0.8447 - val_loss: 0.7051 - val_acc: 0.6712
Epoch 5/5
57/57 [==============================] - 1s 9ms/step
Insert search tags to help users find your model (optional): improved model 2
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 339

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


In [ ]:
# let's see if this helped
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

my_data = data[data['username'] == 'shivanshc']

# Stylize leaderboard data
mycompetition.stylize_leaderboard(my_data)

,accuracy,f1_score,precision,recall,ml_framework,transfer_learning,deep_learning,model_type,depth,num_params,embedding_layers,conv1d_layers,maxpooling1d_layers,simplernn_layers,dropout_layers,flatten_layers,lstm_layers,inputlayer_layers,concatenate_layers,bidirectional_layers,globalmaxpooling1d_layers,globalaveragepooling1d_layers,dense_layers,batchnormalization_layers,sigmoid_act,softmax_act,tanh_act,relu_act,loss,optimizer,memory_size,team,username,version
115,77.83%,77.77%,78.14%,77.83%,keras,nan,True,Sequential,5.000000,174658.000000,1.000000,nan,nan,nan,nan,1.000000,2.000000,nan,nan,nan,nan,nan,1.000000,nan,nan,1.000000,2.000000,nan,str,RMSprop,699936.000000,3.000000,shivanshc,330
236,68.06%,66.93%,70.94%,68.08%,keras,nan,True,Sequential,10.000000,162170.000000,1.000000,2.000000,1.000000,nan,nan,1.000000,nan,nan,nan,nan,1.000000,nan,4.000000,nan,nan,3.000000,nan,2.000000,str,RMSprop,650480.000000,3.000000,shivanshc,335
245,60.15%,53.79%,72.77%,60.19%,keras,nan,True,Sequential,5.000000,174658.000000,1.000000,nan,nan,nan,nan,1.000000,2.000000,nan,nan,nan,nan,nan,1.000000,nan,nan,1.000000,2.000000,nan,str,RMSprop,699936.000000,3.000000,shivanshc,334
246,59.71%,53.02%,72.80%,59.76%,keras,nan,True,Sequential,5.000000,174658.000000,1.000000,nan,nan,nan,nan,1.000000,2.000000,nan,nan,nan,nan,nan,1.000000,nan,nan,1.000000,2.000000,nan,str,RMSprop,699936.000000,3.000000,shivanshc,339
306,50.05%,33.36%,25.03%,50.00%,keras,True,True,Sequential,4.000000,980065.000000,1.000000,nan,nan,nan,nan,1.000000,nan,nan,nan,nan,nan,nan,2.000000,nan,1.000000,nan,nan,1.000000,str,RMSprop,3921008.000000,3.000000,shivanshc,336


### (4) Discuss which models you tried and which models performed better and point out relevant hyper-parameter values for successful models.


I had best success with the Embedding layer and LSTM models, with the increased epoch count.